# BIP's error function

To the error function does expected to receive as input the following pandas.DataFrame struncture:

| StoreID  | Month | Region | NumberOfSales | _NumberOfSales |
| ---------|----------------|---------------| ---------------|
| 1000	   |3      | 4      | 16            | 16             |
| 1000	   |4      | 4      | 30            | 23             |
| 1001	   |3      | 6      | 410           | 411            |
| 1001	   |4      | 27     | 3130          | 3120           |
| 1002	   |3      | 58     | 10            | 8              |

Where:
 
 - *NumberOfSales* are the **test actual values**
 - *_NumberOfSales* are **the predicted values**
 
 

Start from test set to simulate a predicted dataset

In [1]:
from import_man import *

df = pd.read_csv('./dataset/preprocessed_train.csv')

print("Shape before: " + str(df.shape))

# Let's work on a reduced instance of the test set
df = df.sample(n=5000)

print("Shape after: " + str(df.shape))

Shape before: (523021, 51)
Shape after: (5000, 51)


In [2]:
error_evaluation_columns = ['D_Month', 'Region', 'NumberOfSales', '_NumberOfSales']

# Create fake predicted sales 
df['_NumberOfSales'] = df.NumberOfSales.apply(lambda x: df['NumberOfSales'].sample().values[0])


# Save a copy of the dataframe to be passed to the function in order to evaluate it
df_BIP_err_fun_eval = df


# Remove useless columns
df =  df[error_evaluation_columns]

df.head(20)

,D_Month,Region,NumberOfSales,_NumberOfSales
58794,7,3,2973.000000,5669.779242
79109,10,10,3356.000000,6860.000000
120732,7,9,4243.000000,5582.000000
426165,11,0,16128.000000,5185.000000
509998,6,9,6229.000000,9481.000000
505708,8,1,2043.000000,4305.000000
355044,2,0,3640.000000,6189.714992
346851,2,9,5219.000000,6224.000000
77940,2,9,5590.000000,6142.000000
229733,3,3,7083.000000,5609.647986


In [3]:
# let's keep only march and april
# df = df.loc[df['D_Month'].isin([3, 4])] No. let's just evaluate months provided

# compute the difference between actual and predicted NumberOfSales and do the abs
df['abs_diff'] = df.apply(lambda x: abs(x['NumberOfSales'] - x['_NumberOfSales']), axis=1)

df.head(20)

,D_Month,Region,NumberOfSales,_NumberOfSales,abs_diff
58794,7,3,2973.000000,5669.779242,2696.779242
79109,10,10,3356.000000,6860.000000,3504.000000
120732,7,9,4243.000000,5582.000000,1339.000000
426165,11,0,16128.000000,5185.000000,10943.000000
509998,6,9,6229.000000,9481.000000,3252.000000
505708,8,1,2043.000000,4305.000000,2262.000000
355044,2,0,3640.000000,6189.714992,2549.714992
346851,2,9,5219.000000,6224.000000,1005.000000
77940,2,9,5590.000000,6142.000000,552.000000
229733,3,3,7083.000000,5609.647986,1473.352014


In [4]:
# Let's sum over the region
df_sums_by_region = df.groupby(['Region']).sum()
df_sums_by_region.head(20)

,D_Month,NumberOfSales,_NumberOfSales,abs_diff
Region,,,,
0,2522,1.882642e+06,1.925984e+06,9.185545e+05
1,1005,6.972180e+05,7.509468e+05,3.544852e+05
2,3722,3.108599e+06,3.330030e+06,1.405515e+06
3,4336,3.870304e+06,3.068434e+06,1.759728e+06
4,801,7.739244e+05,5.686686e+05,3.777658e+05
5,2188,1.422355e+06,1.656083e+06,7.506521e+05
6,1496,1.011217e+06,1.154099e+06,4.756360e+05
7,2896,2.110053e+06,2.168826e+06,8.936469e+05
8,1041,7.147806e+05,8.096012e+05,2.932612e+05


In [5]:
# Divide the difference between actual and predicted NumberOfSales by the sum of actual
df_sums_by_region['E_r'] = df_sums_by_region['abs_diff'] / df_sums_by_region['NumberOfSales']

df_sums_by_region.head(20)

,D_Month,NumberOfSales,_NumberOfSales,abs_diff,E_r
Region,,,,,
0,2522,1.882642e+06,1.925984e+06,9.185545e+05,0.487907
1,1005,6.972180e+05,7.509468e+05,3.544852e+05,0.508428
2,3722,3.108599e+06,3.330030e+06,1.405515e+06,0.452138
3,4336,3.870304e+06,3.068434e+06,1.759728e+06,0.454674
4,801,7.739244e+05,5.686686e+05,3.777658e+05,0.488117
5,2188,1.422355e+06,1.656083e+06,7.506521e+05,0.527753
6,1496,1.011217e+06,1.154099e+06,4.756360e+05,0.470360
7,2896,2.110053e+06,2.168826e+06,8.936469e+05,0.423519
8,1041,7.147806e+05,8.096012e+05,2.932612e+05,0.410281


In [6]:
# Get the number of regions
N_regions = len(df.Region.unique())

print("Number of regions: {}".format(N_regions))

Number of regions: 11


In [7]:
Error = df_sums_by_region['E_r'].sum() / N_regions

print("BIP error: {}".format(Error))

BIP error: 0.4627666568632817


### Test of the BIP error function implemented

Test that the get_BIP_error function works as expected

In [8]:
from BIP_error import get_BIP_error

df_BIP_err_fun_eval.head()

,StoreID,Date,IsHoliday,IsOpen,HasPromotions,StoreType,AssortmentType,NearestCompetitor,Region,NumberOfCustomers,...,StoreType_ShoppingCenter,AssortmentType_General,AssortmentType_WithNFDept,AssortmentType_WithFishDept,Events_Fog,Events_Hail,Events_Rain,Events_Snow,Events_Thunderstorm,_NumberOfSales
58794,1084,14/07/2016,0,1,0,Hyper Market,General,1627,3,172.0,...,0,1,0,0,0,0,1,0,0,5669.779242
79109,1113,19/10/2017,0,1,0,Hyper Market,With Non-Food Department,43854,10,236.0,...,0,0,1,0,0,0,1,0,0,6860.000000
120732,1173,18/07/2017,0,1,1,Hyper Market,With Non-Food Department,150,9,339.0,...,0,0,1,0,1,0,1,0,0,5582.000000
426165,1611,07/11/2016,0,1,1,Hyper Market,General,370,0,1054.0,...,0,1,0,0,0,0,1,0,0,5185.000000
509998,1730,05/06/2017,0,1,1,Hyper Market,General,4227,9,356.0,...,0,1,0,0,0,0,0,0,0,9481.000000


In [9]:
# use the function
error = get_BIP_error(df_BIP_err_fun_eval)

Number of regions identified: 11
BIP total error: 0.4627666568632817
